In [1]:
import numpy as np

In [19]:
class Tensor(object):
    def __init__(self,data, autograd=False, creators=None, creation_op=None, id=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if (id is None):
            id = np.random.randint(0,100000)
        self.id = id
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] =1
                else:
                    c.children[self.id] +=1
                    
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt!=0):
                return False
            return True
        
    def backward(self, grad, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception('cannot backprop more than once')
                else:
                    self.children[grad_origin.id] -=1
            if(self.grad is True):
                self.grad = grad
            else:
                self.grad += grad
            
            if(self.all_children_grads_accounted_for() or grad_origin is None):
                if(self.creation_op == 'add'):
                    self.creators[0].backward(self.grad,self)
                    self.creators[1].backward(self.grad,self)
        
    def __add__(self,other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op='add')
        return Tensor(self.data+other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

In [3]:
x = Tensor([1,2,3,4,5])
print(x)

[1 2 3 4 5]


In [4]:
y = x + x
print(y)

[ 2  4  6  8 10]


In [5]:
z = x+y

In [6]:
print(z)

[ 3  6  9 12 15]


In [7]:
z.backward(Tensor(np.array([1,1,1,1,1])))

In [8]:
print(x.grad)

[1 1 1 1 1]


In [9]:
z.creators

[array([1, 2, 3, 4, 5]), array([ 2,  4,  6,  8, 10])]

In [11]:
print(y.grad)

[1 1 1 1 1]


In [12]:
print(z.creation_op)

add


In [20]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a + b
f = c + d
g = e + f

In [21]:
g.backward(Tensor(np.array([1,1,1,1,1])))

In [22]:
print(a.grad)

None


In [24]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True        
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [25]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a+b
e = b+c
f = d+e
f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))

AttributeError: 'NoneType' object has no attribute 'data'

In [18]:
print(b.grad.data)

[1 1 1 1 1]
